In [1]:
cd src

/home/jupyter/MedClip/src


In [2]:
%load_ext tensorboard

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from preprocess import pipelines
import datetime

In [3]:
df=pd.read_csv('../data/intermediate/inter_chexpert.csv')

In [4]:
data_pipeline=pipelines.make_pipeline('../data/intermediate/inter_chexpert.csv',downscale=False)

making input pipeline


2022-07-08 00:44:53.369795: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 00:44:53.424050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 00:44:53.424937: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-08 00:44:53.426682: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
data_pipeline

{'CLIP': {'train': <ZipDataset element_spec=({'image': TensorSpec(shape=(299, 299, 3), dtype=tf.float32, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)},)>,
  'val': <ZipDataset element_spec=({'image': TensorSpec(shape=(299, 299, 3), dtype=tf.float32, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)},)>,
  'test': <ZipDataset element_spec=({'image': TensorSpec(shape=(299, 299, 3), dtype=tf.float32, name=None), 'text': TensorSpec(shape=(), dtype=tf.string, name=None)},)>},
 'captioning': {'train': <ZipDataset element_spec=(TensorSpec(shape=(299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>,
  'val': <ZipDataset element_spec=(TensorSpec(shape=(299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>,
  'test': <ZipDataset element_spec=(TensorSpec(shape=(299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>},
 'labeling': {'t

In [6]:

#High level args
data_pipeline=pipelines.make_pipeline('../data/intermediate/inter_chexpert.csv',downscale=False)
embedding_size = 128
dataset = data_pipeline['labeling']

train_backbone= False
log_dir='../model_logs/pretraining/classification/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_filepath='../model_checkpoints/pretraining/classification/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Inside function



train_data=data_pipeline['labeling']['train'].batch(128)
val_data=data_pipeline['labeling']['val'].batch(128)

input_shape = train_data.element_spec[0].shape.as_list()[1:]
n_classes = int(train_data.element_spec[1].shape.as_list()[1])


inputs=tf.keras.Input(shape=input_shape, name='input')




backbone= tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    pooling=None,
)





if (train_backbone==False):
    for layer in backbone.layers:
        layer.trainable=False
x = tf.keras.applications.resnet50.preprocess_input(inputs) # preprocess data
x = backbone(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(embedding_size,activation=None,name='encoding_layer')(x)
x = tf.keras.layers.Dense(n_classes,activation='sigmoid',name='classification_layer')(x)

model=tf.keras.Model(inputs=inputs,outputs=x)


# Define callbacks
tf_callback=tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,
        write_graph=True,
        write_images=False,
        write_steps_per_second=False,
        update_freq='epoch',
        profile_batch=0,
        embeddings_freq=0,
        embeddings_metadata=None,
    )

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)




model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=['accuracy','AUC','Precision','Recall']
             )
model.summary()

making input pipeline
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 299, 299, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 299, 299, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 299, 299, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 10, 10, 2048)      23587712  
                                                                 
 flatten (Flatten)           (None, 204800)            0         
                                                                 
 encoding_layer (Dense)      (None, 128)               26214528  
                                       

In [ ]:
model.fit(train_data,epochs=20,callbacks=[tf_callback],validation_data=val_data)

Epoch 1/20


2022-07-08 00:45:04.534437: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200


 212/1224 [====>.........................] - ETA: 39:18 - loss: 5.0637 - accuracy: 0.7213 - auc: 0.8336 - precision: 0.7505 - recall: 0.7506

In [ ]:
%tensorboard --logdir ../model_logs/pretraining/